In [11]:
import sys
sys.path.insert(0, '../src/libs')
sys.path.insert(0, '../src/numpy_path')

In [3]:
# our file imports
from music_transformer import *
from transformer_training_helpers import *
from song_classification import *
from dataset import *
from midi2audio import FluidSynth

# musicautobot
from musicautobot.numpy_encode import *
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.utils.midifile import *
from musicautobot.utils.file_processing import process_all

In [4]:
myDs = MidiDataset('../numpy_path', 150, 'N/A', '.npy')
dataloader = DataLoader(myDs, 1, shuffle=True)

for _, item in enumerate(dataloader):
    print(item)
    break

tensor([[[  0.,   1.,   8.,  ..., 160.,  63., 161.],
         [ 47., 140.,   8.,  ..., 140.,  63., 140.],
         [ 59., 140.,   8.,  ..., 139.,  66., 140.],
         ...,
         [ 76., 139.,  71.,  ..., 138.,  78., 140.],
         [ 75., 138.,  42.,  ..., 140.,  73., 138.],
         [ 68., 138.,  47.,  ..., 139.,  83., 138.]]])


In [18]:
print(len(myDs))

9636


In [15]:
def topk_decode(model, src, src_mask, max_len, start_symbol, k):
  memory = model.encode(src, src_mask)
  ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
  for i in range(max_len-1):
    out, w2 = model.decode(memory, src_mask,
                       Variable(ys),
                       Variable(subsequent_mask(ys.size(1))
                                .type_as(src.data)))
    prob = model.generator(out[:, -1])
    _, next_words = torch.topk(prob, k = k, dim = 1)
    # if starting, then create 
    #print(next_words)
    if i == 0:
      next_word = next_words[0][0].data.item()
    else:
      k2 = random.randint(0,k-1)
      next_word = next_words[0][k2].data.item()
    #print(next_word)
    #k = random.randint(0,4)
    #next_word = next_words[k].data[0]
    ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
  
  return ys

def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out, w2 = model.decode(memory, src_mask, 
                           Variable(ys), 
                           Variable(subsequent_mask(ys.size(1))
                                    .type_as(src.data)))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1) # change this for top k
        next_word = next_word.data[0]
        ys = torch.cat([ys, 
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys

In [16]:
myModel.eval()
# ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
#src = Variable(torch.LongTensor([[1,2,3,4,5,6,7,8,9,10]]))
#src = Variable(torch.LongTensor([[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]])).cuda()
idx = 1
src = myDs[idx].reshape(myDs[idx].shape[0] * myDs[idx].shape[1])[:100].unsqueeze(dim=0).long().cuda() # all_data.valid_ds[1][0].to_tensor()[:100].unsqueeze(dim=0)
#print(src.size())
src_mask = Variable(torch.ones(1, 1, 100)).unsqueeze(dim=0).cuda() # 26 was 10
output = greedy_decode(myModel.cuda(), src, src_mask, max_len=100, start_symbol=0)
output2 = topk_decode(myModel.cuda(), src, src_mask, max_len=100, start_symbol=0, k=2)

#print(output2[0])
#print(output[0])

output.cuda()
output2.cuda()

print("GREEDY: ", output[0])
print("TOPK: ", output2[0])
print("SRC: ", src[0])

ncorrect = 0
for i in range(len(src)):
  if src[0][i] == output2[0][i]:
    ncorrect += 1
print("TOPK: ", ncorrect / len(src[0]))

ncorrect = 0
for i in range(len(src)):
  if src[0][i] == output[0][i]:
    ncorrect += 1
print("GREEDY: ", ncorrect / len(src[0]))

NameError: name 'myModel' is not defined

In [ ]:
%matplotlib inline
from music21 import *
import fluidsynth
#environment.set()
#print(output)
output[0, 1] = 1
output2[0, 1] = 1
#print(src.cpu().numpy())
temp_vocab = MusicVocab.create()
#print(temp_vocab.stoi)
MI = temp_vocab.to_music_item(src.cpu().numpy()[0])
MO = temp_vocab.to_music_item(output.cpu().numpy()[0])
MT = temp_vocab.to_music_item(output2.cpu().numpy()[0])
print("SRC.SHP: ", src.shape)
print("OUTPUT.SHP: ", output.shape)
print("OUTPUT2.SHP: ", output2.shape)
print(src)
print(output)
print(output2)
#print(MI.to_text())
#print(MO.to_text())
#print(MT.to_text())

#new_tens = myDs[0].reshape(myDs[0].shape[0] * myDs[0].shape[1])

MMM = temp_vocab.to_music_item(new_tens.numpy().astype(int))
MI.stream.write('midi', fp='./temp_outputs/inp.mid')
MO.stream.write('midi', fp='./temp_outputs/pred.mid')
MT.stream.write('midi', fp='./temp_outputs/topk.mid')
#MMM.stream.write('midi', fp='./temp_outputs/inp_test.mid')

In [14]:
import os
for filename in os.listdir('/home/trevor/Desktop/MusGen/src/numpy_path'):
    

['TRWITQS128F147B533.npy',
 'TRELDPP128E07967DF.npy',
 'TRBGGMW128F422401E.npy',
 'TRKEUKU128F1452C02.npy',
 'TRFHGSV128F4273973.npy',
 'TRGJMAG12903CE48E3.npy',
 'TRWEINZ128F4271C7B.npy',
 'TRSOBXT128F1459EB4.npy',
 'TRHGVWD128F92E1C6F.npy',
 'TRZYBWV128F426E29F.npy',
 'TRETOTR12903CBC27D.npy',
 'TRUCIKF128F4291BBF.npy',
 'TRHEQIF128F92DCF1B.npy',
 'TRJBTGZ128F42947F1.npy',
 'TRPEOEU128F931DE86.npy',
 'TRLWTAG128E0789037.npy',
 'TRBZHPO128F147EFD3.npy',
 'TRCHHHC128F428A398.npy',
 'TRMYFXB128F92DB31F.npy',
 'TRVGGYU128F930A9FA.npy',
 'TRNHOTA128E078B309.npy',
 'TRJVDQK128F4273C92.npy',
 'TRUHUYY128F423D6ED.npy',
 'TRZWSFR128F933209B.npy',
 'TROIJLZ128F92F820B.npy',
 'TRZHWKE128F92E27B1.npy',
 'TRCKNGE128F92DA3F3.npy',
 'TREDEOB128F92CFF26.npy',
 'TRVYVRC128F9324F4E.npy',
 'TRBTJNG128F9306659.npy',
 'TRANJGV128F930818A.npy',
 'TRSUTUD128F42333F4.npy',
 'TRFDXTG128F1496037.npy',
 'TRXXPYK128F14805B7.npy',
 'TRDYSOG128F9343C43.npy',
 'TRCFNZR12903CD78B9.npy',
 'TRQZNZN12903CF8867.npy',
 